<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Consolidaci%C3%B3n_de_pedidos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalaciones previas

In [113]:
#!pip install xlsxwriter
#!pip install python-calamine
#!pip install fastexcel

# Librerías necesarias

In [114]:
import polars as ps
import pandas as pd
import numpy as np

# Archivos necesarios

In [115]:
renglonaje_og = ps.read_excel("Renglonaje.xlsx", sheet_id=1)
renglonaje = renglonaje_og.select(['código', 'clave', 'Total que paga Edy a Truper contando reserva en garantía CON IVA'])
renglonaje = renglonaje.rename({'Total que paga Edy a Truper contando reserva en garantía CON IVA':'Costo', 'código':'clave', 'clave':'sku'})

## Relación Asín-SKU

In [117]:
asin = ps.read_excel('Asin.xlsx', sheet_id=2)
asin = asin.rename({'sku':'alfa', 'Sku Truper':'sku'})
asin = asin.select(['sku', 'alfa', 'asin'])
# Conditionally update the 'alfa' column:
# - Use with_columns to modify existing columns.
# - ps.when checks if the value in 'alfa' is equal to the value in 'sku'.
# - .then(ps.lit('')) replaces the value with an empty string if the condition is true.
# - .otherwise(ps.col('alfa')) keeps the original 'alfa' value if the condition is false.
# - .alias('alfa') renames the resulting column back to 'alfa'.
asin = asin.with_columns(
    ps.when(ps.col('alfa') == ps.col('sku'))
    .then(ps.lit(''))
    .otherwise(ps.col('alfa'))
    .alias('alfa')
)

sku,alfa,asin
str,str,str
"""ASP-5S""","""06-L62H-WPL2""","""B09Z3BWF34"""
"""EST-29""","""0F-ZVA0-7RNO""","""B013QH5BBG"""
"""LAGAS-3300""","""0J-XTPT-HVJH""","""B084YS54G6"""
"""SCI-14E""","""0W-ZPBA-0RDL""","""B013R4OSRG"""
"""ESMU-112""","""100495""","""B08DMBSGWK"""
…,…,…
"""ZP-5MP""","""""","""B00UY22SFE"""
"""ZP-5MX""","""""","""B00UY2669S"""
"""ZP-5P""","""""","""B013V088V8"""


In [118]:
renglonaje = renglonaje.join(asin, on='sku', how='left')

In [119]:
renglonaje = renglonaje.filter(ps.col('Costo').is_not_null())
renglonaje = renglonaje.fill_null('')

clave,sku,Costo,alfa,asin
i64,str,f64,str,str
100048,"""PET-15X""",290.9246,"""""",""""""
100049,"""PET-15C""",407.29444,"""""","""B084Z6ZF55"""
100098,"""REP-CUT-5X""",13.96797,"""""","""B084Z4PX65"""
100099,"""REP-CUT-6X""",24.8253,"""""","""B084Z4VT1K"""
100101,"""REP-CUT-650""",65.94158,"""""",""""""
…,…,…,…,…
44814,"""CLE-1-1/4B""",44.64566,"""""","""B013SFAZA8"""
44815,"""CLE-1-1/2B""",44.64566,"""""","""B013SFB17O"""
47389,"""TC-1-1/4-GC""",103.25929,"""""",""""""


In [120]:
c_sku = renglonaje.select(['sku', 'Costo'])
c_sku = c_sku.rename({'Costo':'Costo_sku'})
c_alfa = renglonaje.select(['alfa', 'Costo'])
c_alfa = c_alfa.rename({'Costo':'Costo_alfa', 'alfa':'sku'})
c_asin = renglonaje.select(['asin', 'Costo'])
c_asin = c_asin.rename({'Costo':'Costo_asin', 'asin':'sku'})
c_clave = renglonaje.select(['clave', 'Costo'])
c_clave = c_clave.rename({'Costo':'Costo_clave', 'clave':'sku'})
c_clave = c_clave.with_columns(ps.col('sku').cast(ps.Utf8))

# Reportes de Amazon
Aquí se definen cuantos reportes y transacciones se van a procesar

In [153]:
loop = True
while loop:
  try:
    reportes_cantidad = input("Cantidad de reportes: ")
    reportes_cantidad = int(reportes_cantidad)
    loop = False
  except:
    print("\nIngresa un número válido\n")

Cantidad de reportes: 3


In [151]:
loop = True
while loop:
  try:
    transacciones_cantidad = input("Cantidad de transacciones: ")
    transacciones_cantidad = int(transacciones_cantidad)
    loop = False
  except:
    print("\nIngresa un número válido\n")

Cantidad de transacciones: s
Ingresa un número válido
Cantidad de transacciones: w
Ingresa un número válido
Cantidad de transacciones: q
Ingresa un número válido
Cantidad de transacciones: 57


# Leyendo Reportes

Si solo se va a leer un reporte, este deberá ser guardado como "Reporte". Si no, los reportes deberán ser guardados de forma seriada, es decir, "Reporte1", "Reporte2", "Reporte3", etc.

El orden no debería importar

In [123]:
archivo = ''
archivos = []
if reportes_cantidad == 1:
    archivo = "Reportes/Reporte.txt"
else:
  for i in range(reportes_cantidad):
      archivos.append((f"Reportes/Reporte{i+1}.txt"))

In [124]:
if archivo == '':
  reportes = ps.read_csv(archivos[0], separator="\t")
else:
  reportes = ps.read_csv(archivo, separator="\t")

In [125]:
if len(archivos) > 1:
  for i in range(1, reportes_cantidad):
      reportes = reportes.vstack(ps.read_csv(archivos[i], separator="\t", schema=reportes.schema, truncate_ragged_lines=True))
  print(f"Uniendo {reportes_cantidad} reportes")


Uniendo 3 reportes


# Leyendo Transacciones

Si solo se va a leer una transacción, esta deberá ser guardada como "Transacciones". Si no, deberán ser guardadas de forma seriada, como los reportes.

***Nota: Los reportes deberán ser pasados primero por la herramienta de limpieza de transacciones. Si no, generarán problemas***

In [126]:
archivo = ''
archivos = []
if transacciones_cantidad == 1:
    archivo = "Transacciones/Transacciones.csv"
else:
  for i in range(transacciones_cantidad):
      archivos.append((f"Transacciones/Transacciones{i+1}.csv"))

In [127]:
if archivo == '':
  transacciones = ps.read_csv(archivos[0], separator=",")
else:
  transacciones = ps.read_csv(archivo, separator=",")

In [128]:
if len(archivos) > 1:
  for i in range(1, transacciones_cantidad):
      transacciones = transacciones.vstack(ps.read_csv(archivos[i], separator = ',', schema=transacciones.schema, truncate_ragged_lines=True))

Renombramos la columna de Número de pedido a amazon-order-id, para hacer la unión luego con el reporte de pedidos

In [129]:
transacciones = transacciones.rename({'Número de pedido':'amazon-order-id'})

In [130]:
dfr = reportes.join(transacciones, on='amazon-order-id', how='left', suffix='_transacciones')

In [131]:
formato_num = {ps.Int64:"0", ps.Float64:"0.00"}

In [132]:
dft = transacciones.join(reportes, on='amazon-order-id', how='left', suffix='_reportes')

# Consolidando reporte de envíos

In [133]:
archivo_envios = 'Envios enero-julio.xlsx'
envios_1 = pd.read_excel(archivo_envios, sheet_name = 0, header=3)
envios_2 = pd.read_excel(archivo_envios, sheet_name = 1, header=3)
reporte_envios = pd.concat([envios_1, envios_2])
reporte_envios = reporte_envios.reset_index(drop=True)
reporte_envios.to_excel("Envios.xlsx", index=False)

In [134]:
envios = ps.read_excel("Envios.xlsx", sheet_id=1)

In [135]:
envios = envios.with_columns(ps.col('PROTECCION ($)').fill_null(0))
envios = envios.with_columns(ps.col('PROTECCION ($)').cast(ps.Float32))
envios.schema

Schema([('NUMERO ORDEN', String),
        ('GUÍA', String),
        ('PAQUETERIA', String),
        ('SERVICIO', String),
        ('FECHA DE CREACION', Date),
        ('NOMBRE DESTINO', String),
        ('CP DESTINO', Int64),
        ('VALOR GUIA ($)', Float64),
        ('ZONA EXTENDIDA ($)', Float64),
        ('EXTRAGRANDE ($)', Int64),
        ('SOBREPESOS ($)', Int64),
        ('TOTAL GUIA ($)', Float64),
        ('ESTATUS RASTREO', String),
        ('PROTECCION ($)', Float32)])

# Juntando el reporte de envíos y las transacciones

In [136]:
envios = envios.rename({'NUMERO ORDEN':'amazon-order-id'})
envios = envios.select(['amazon-order-id', 'VALOR GUIA ($)', 'ZONA EXTENDIDA ($)', 'EXTRAGRANDE ($)', 'SOBREPESOS ($)', 'PROTECCION ($)', 'TOTAL GUIA ($)'])
dfr = dfr.join(envios, on='amazon-order-id', how='left')

In [137]:
dfr = dfr.with_columns(ps.col('VALOR GUIA ($)').fill_null(0),
                       ps.col('ZONA EXTENDIDA ($)').fill_null(0),
                       ps.col('EXTRAGRANDE ($)').fill_null(0),
                       ps.col('SOBREPESOS ($)').fill_null(0),
                       ps.col('PROTECCION ($)').fill_null(0),
                       ps.col('TOTAL GUIA ($)').fill_null(0))

In [138]:
dft = dft.join(envios, on='amazon-order-id', how='left')
dft = dft.with_columns(ps.col('VALOR GUIA ($)').fill_null(0),
                       ps.col('ZONA EXTENDIDA ($)').fill_null(0),
                       ps.col('EXTRAGRANDE ($)').fill_null(0),
                       ps.col('SOBREPESOS ($)').fill_null(0),
                       ps.col('PROTECCION ($)').fill_null(0),
                       ps.col('TOTAL GUIA ($)').fill_null(0))

In [139]:
dfr = dfr.join(c_sku, on='sku', how='left')
dfr = dfr.join(c_alfa, on='sku', how='left')
dfr = dfr.join(c_asin, on='sku', how='left')
dfr = dfr.join(c_clave, on='sku', how='left')

In [140]:
dfr = dfr.with_columns(ps.col('Costo_sku').fill_null(0),
                       ps.col('Costo_alfa').fill_null(0),
                       ps.col('Costo_asin').fill_null(0),
                       ps.col('Costo_clave').fill_null(0))

In [141]:
dfr = dfr.with_columns((ps.col('Costo_sku') + ps.col('Costo_alfa') + ps.col('Costo_asin') + ps.col('Costo_clave')).alias('Costo Truper'))

In [142]:
dfr = dfr.drop(['Costo_sku', 'Costo_alfa', 'Costo_asin', 'Costo_clave'])

In [143]:
dfr.write_excel('Reporte de envios.xlsx', worksheet="Reportes")

In [144]:
dft = dft.join(c_sku, on='sku', how='left')
dft = dft.join(c_alfa, on='sku', how='left')
dft = dft.join(c_asin, on='sku', how='left')
dft = dft.join(c_clave, on='sku', how='left')

In [145]:
dft = dft.with_columns(ps.col('Costo_sku').fill_null(0),
                       ps.col('Costo_alfa').fill_null(0),
                       ps.col('Costo_asin').fill_null(0),
                       ps.col('Costo_clave').fill_null(0))

In [146]:
dft = dft.with_columns((ps.col('Costo_sku') + ps.col('Costo_alfa') + ps.col('Costo_asin') + ps.col('Costo_clave')).alias('Costo Truper'))

In [147]:
dft = dft.drop(['Costo_sku', 'Costo_alfa', 'Costo_asin', 'Costo_clave'])

In [148]:
dft.write_excel('Reporte de transacciones.xlsx', dtype_formats={ps.Float64():'General'}, worksheet="Transacciones")

In [149]:
dft.head(4)

Fecha,Tipo de transacción,amazon-order-id,Detalles del producto,Total de cargos del producto,Total de reembolsos promocionales,Tarifas de Amazon,Otros,Total (MXN),merchant-order-id,purchase-date,last-updated-date,order-status,fulfillment-channel,sales-channel,order-channel,url,ship-service-level,product-name,sku,asin,item-status,quantity,currency,item-price,item-tax,shipping-price,shipping-tax,gift-wrap-price,gift-wrap-tax,item-promotion-discount,ship-promotion-discount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,payment-method-details,item-extensions-data,is-business-order,purchase-order-number,price-designation,customized-url,customized-page,is-replacement-order,is-exchange-order,original-order-id,is-transparency,is-buyer-requested-cancellation,buyer-requested-cancel-reason,buyer-identification-number,buyer-identification-type,VALOR GUIA ($),ZONA EXTENDIDA ($),EXTRAGRANDE ($),SOBREPESOS ($),PROTECCION ($),TOTAL GUIA ($),Costo Truper
str,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,f64,f64,f64,str,str,str,str,f64,str,str,i64,str,str,str,str,bool,str,str,str,str,bool,bool,str,bool,bool,str,str,str,f64,f64,i64,i64,f32,f64,f64
"""3/5/2025""","""Saldo no disponible""","""---""","""Monto de reserva actual""",0.0,0.0,0.0,-143521.4,-143521.4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0,0,0.0,0.0,0.0
"""3/5/2025""","""Pago del pedido""","""701-5579061-5906663""","""Truper ESE-20, Escalera de ext…",3103.45,0.0,-648.0,496.55,2952.0,null,"""2025-05-03T18:53:57+00:00""","""2025-05-03T23:02:27+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper ESE-20, Escalera de ext…","""16027""","""B013R4N3M2""","""Shipped""",1,"""MXN""",3600.0,496.55,null,null,null,null,null,null,"""GUADALAJARA""","""JALISCO""",44330,"""MX""",null,"""CreditCard""",null,false,null,null,null,null,false,false,null,false,false,null,null,null,376.0,0.0,0,0,0.0,376.0,2327.41674
"""3/5/2025""","""Servicios de envío adquiridos …","""702-8036310-0469852""","""Facturación""",-180.32,0.0,0.0,-159.79,-340.11,null,"""2025-05-03T21:09:18+00:00""","""2025-05-03T22:53:24+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper ESE-24, Escalera de ext…","""ESE-24""","""B013R4N818""","""Shipped""",1,"""MXN""",4250.0,586.21,null,null,null,null,null,null,"""SANCTORUM DE LAZARO CARDENAS""","""TLAXCALA""",90230,"""MX""",null,"""CreditCard""",null,false,null,null,null,null,false,false,null,false,false,null,null,null,0.0,0.0,0,0,0.0,0.0,2754.1128
"""3/5/2025""","""Pago del pedido""","""702-8036310-0469852""","""Truper ESE-24, Escalera de ext…",3663.79,0.0,-765.0,586.21,3485.0,null,"""2025-05-03T21:09:18+00:00""","""2025-05-03T22:53:24+00:00""","""Shipped""","""Merchant""","""Amazon.com.mx""","""WebsiteOrderChannel""",null,"""Standard""","""Truper ESE-24, Escalera de ext…","""ESE-24""","""B013R4N818""","""Shipped""",1,"""MXN""",4250.0,586.21,null,null,null,null,null,null,"""SANCTORUM DE LAZARO CARDENAS""","""TLAXCALA""",90230,"""MX""",null,"""CreditCard""",null,false,null,null,null,null,false,false,null,false,false,null,null,null,0.0,0.0,0,0,0.0,0.0,2754.1128
